In [1]:
# MarkedLM 테스트 예제
#
# => input_ids : [CLS][MASK]문장[SEP]
# => attention_mask : 1111111111
# => token_type_ids : 0000000


import numpy as np
import torch
import torch.nn.functional as F
import os
import sys
from transformers import BertTokenizer, BertForMaskedLM

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="bertMLMtest", logfilname="bertMLMtest")
device = GPU_info()
seed_everything(111)

logfilepath:bwdataset_2022-03-16.log
logfilepath:qnadataset_2022-03-16.log
logfilepath:bertMLMtest_2022-03-16.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
#############################################################################################
# 변수들 설정
# - model_path : from_pretrained() 로 호출하는 경우에는 모델파일이 있는 폴더 경로나 
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')
#          torch.load(model)로 로딩하는 경우에는 모델 파일 풀 경로
#
# - vocab_path : from_pretrained() 호출하는 경우에는 모델파일이 있는 폴더 경로나
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')   
#          BertTokenizer() 로 호출하는 경우에는 vocab.txt 파일 풀 경로,
#
# - OUTPATH : 출력 모델, vocab 저장할 폴더 경로
#############################################################################################

model_path = 'model/bert-multilingual/bmc_fpt_kowiki20200920.train_model_0225'
vocab_path = 'model/bert-multilingual/bmc_fpt_kowiki20200920.train_model_0225'

# tokeniaer 및 model 설정
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# strip_accents=False : True로 하면, 가자 => ㄱ ㅏ ㅈ ㅏ 식으로 토큰화 되어 버림(*따라서 한국어에서는 반드시 False)
# do_lower_case=False : # 소문자 입력 사용 안함(한국어에서는 반드시 False)
tokenizer = BertTokenizer.from_pretrained(vocab_path, strip_accents=False, do_lower_case=False) 
   
model = BertForMaskedLM.from_pretrained(model_path)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(143772, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [3]:
model.num_parameters()

196603548

In [4]:
# tokenier 테스트
print(len(tokenizer))
print(tokenizer.encode("눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다", "정말 재미있다"))
print(tokenizer.convert_ids_to_tokens(131027))
print(tokenizer.convert_tokens_to_ids('정말'))

143772
[101, 9034, 10530, 119728, 11018, 128441, 10739, 69708, 42428, 10459, 10020, 12030, 28143, 10892, 124227, 12508, 49137, 102, 122108, 131027, 11903, 102]
재미있
122108


In [65]:
#text = ["한국의 수도는 [MASK] 이다"]
#text = ["에펠탑은 [MASK]에 있다"]
text = ['한국의 수도는 [MASK] 이다', '에펠탑은 [MASK]에 있다', '충무공 이순신은 [MASK]에 최고의 장수였다']

In [66]:
max_length = 128
tokenized_input = tokenizer(text, max_length=max_length, truncation=True, padding='max_length', return_tensors='pt')

In [67]:
print(tokenized_input)

mask_idx_list = []
for tokens in tokenized_input['input_ids'].tolist():
    #print(tokens)
    token_str = [tokenizer.convert_ids_to_tokens(s) for s in tokens]
    print(token_str)
    
    # **위 token_str리스트에서 [MASK] 인덱스를 구함
    # => **해당 [MASK] 안덱스 값 mask_idx 에서는 아래 출력하는데 사용됨
    mask_idx = token_str.index('[MASK]')
    mask_idx_list.append(mask_idx)
    print(f'[MASK] idx : {mask_idx}')
    
print(mask_idx_list)   


{'input_ids': tensor([[   101,  48556,  10459,  69283,  11018,    103,  30919,    102,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              

In [68]:

outputs = model(**tokenized_input)

# 출력은 [1, 256, 143772] 처럼 [batch, token수, vocab수]로 출력되고,
# 여기서 각 toekn수 마다 vocab의 확률이 반환됨.
# => 각 token 중 [MASK] 토근의 vockab이 최대 확률을 구하면 됨
logits = outputs.logits
print(logits.shape)

torch.Size([3, 128, 143772])


In [69]:
for idx, mask_idx in enumerate(mask_idx_list):
    
    # logits_pred 값은 [token수]로 출력됨
    logits_pred=torch.argmax(F.softmax(logits[idx]), dim=1)
    #print(logits_pred.shape)
  
    # logits에서 [MASK]에 token_id 를 구함
    mask_logits_idx = int(logits_pred[mask_idx])

    # [MASK]에 해당하는 tokeni 구함
    mask_logits_token = tokenizer.convert_ids_to_tokens(mask_logits_idx)

    # 결과 출력 
    print('\n')
    print('*Input: {}'.format(text[idx]))
    print('*[MASK] : {} ({})'.format(mask_logits_token, mask_logits_idx))




*Input: 한국의 수도는 [MASK] 이다
*[MASK] : 서울 (48253)


*Input: 에펠탑은 [MASK]에 있다
*[MASK] : 파리 (120425)


*Input: 충무공 이순신은 [MASK]에 최고의 장수였다
*[MASK] : 임진왜란 (123342)


/tmp/ipykernel_49907/1193083005.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits_pred=torch.argmax(F.softmax(logits[idx]), dim=1)
